In [1]:
import pandas as pd
import os
import re
import warnings
from decimal import Decimal#保证比例呈现两位小数
import sys
import numpy as np
from datetime import datetime
from docx import Document
from docx.enum.style import WD_STYLE_TYPE
import pymssql
from docx.enum.text import WD_ALIGN_PARAGRAPH  #设置对象居中、对齐等。
from docx.enum.text import WD_TAB_ALIGNMENT,WD_TAB_LEADER,WD_LINE_SPACING  #设置制表符等
from docx.shared import Inches   #设置图像大小
from docx.shared import Pt,Cm   #设置像素、缩进等
from docx.shared import RGBColor    #设置字体颜色
from docx.shared import Length    #设置宽度
from docx.oxml.ns import qn  #设置中文版式
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_ALIGN_VERTICAL #垂直居中
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime
from docx.table import _Cell
from docx.oxml import OxmlElement
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
import functools
import traceback
import time
import requests
warnings.filterwarnings('ignore')

# 读取excel

In [2]:
try:
    excel_path = '数据'
    model_path = '模板'
    file_names = os.listdir(excel_path)
    file_names = [file for file in file_names if file.endswith(('.xlsx','.xls','.XLSX','.XLS'))]

    data_group=[]
    for file in file_names:
        file_path = os.path.join(excel_path, file)  # 构建文件的完整路径
        df = pd.read_excel(file_path)
        data_group.append(df)

    key_group={}
    for index in range(len(data_group)):
        key_group[data_group[index].columns[0]]=index
        
    print('程序读取excel成功！')
except:
    print('程序读取excel报错，错误信息如下：---------------------------')
    traceback.print_exc()
    time.sleep(5)
    raise

程序读取excel成功！


# 自定义函数

In [3]:
# 取data的前三，可自动忽略汇总行和总计行，忽略非数字,不忽略百分数
def get_3rd(data,num,not_3=3):#num为倒序排序的依据,不取前三也行
    datat=data.copy()
    datat= datat[(datat.iloc[:,0] != '汇总') & (datat.iloc[:,0] != '总计')& (datat.iloc[:,0] != 'left') & (datat.iloc[:,1] != '汇总') & (datat.iloc[:,1] != '总计')]
    datat['temp']=data.apply(lambda x:(float(re.sub('%','',x[num])) if '%' in x[num] else 0) if isinstance(x[num],str) else x[num],axis=1)
    datat=datat.sort_values(by='temp',ascending=False).reset_index(drop=True).drop(['temp'],axis=1)
    return [datat.loc[x].tolist() for x in range(datat.shape[0]>not_3 and not_3 or datat.shape[0])]

In [4]:
def mssql_data(sql,lua=[]):
#     conn = pymssql.connect(server='39.104.209.155', user='sa', password='admin_0', database='hn_szx')
    conn = pymssql.connect(server='172.16.4.175', user='sa', password='SAadmin_123', database='hn_szx')
    cursor = conn.cursor()
    cursor.execute(sql)
    data = cursor.fetchall()
    data=pd.DataFrame(data, columns=[column[0] for column in cursor.description])
    for i in lua:
        data[i]=data[i].apply(lambda x:x.encode('latin1').decode('gbk'))
    cursor.close()
    conn.close()
    return data

In [5]:
def add_p(content1,content2):
    global document
    document.add_paragraph(content1, style='CustomStyle2')
    document.add_paragraph(content2, style='CustomStyle1')

In [6]:
def xy(datat,x,x1,y):
    return datat[datat[x]==x1][y].values[0]

def change_to_decimal(x):
    return  str(Decimal(x*100).quantize(Decimal("0.00")))+'%'


In [7]:
def Set_Background_Color(cell,rgbColor):
    shading_elm = parse_xml(r'<w:shd {} w:fill="{color_value}"/>'.format(nsdecls('w'),color_value = rgbColor)) #固定写法，照抄即可
    cell._tc.get_or_add_tcPr().append(shading_elm)
    
def add_table(datat,tail=[]):#自动加首行
    data=datat.copy()
    cols=data.columns
    #总计行
    if len(tail)==data.shape[1]:
        dict2={cols[i]:[tail[i]] for i in range(len(cols))}
        datatt=pd.DataFrame(dict2)
        data=pd.concat([data,datatt],ignore_index=True)
    global document
    row_count = data.shape[0]
    col_count = data.shape[1]
    table = document.add_table(rows=row_count+1, cols=col_count,style='table1')#table1只是保证有边框
        
    table.autofit = True #自动调整宽度和高度
    
    # 单元格样式：居中对齐，边框颜色为黑色，仿宋_GB2312小四字体
    cell_format = table.style.paragraph_format
    cell_format.alignment = WD_ALIGN_VERTICAL.CENTER   # 垂直居中对齐
    
    # 填充数据
    for i, column_name in enumerate(data.columns):
        cell = table.cell(0, i)
        cell.text = column_name
        cell.paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
        cell.paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        # 设置标题四号黑体字体
        cell.paragraphs[0].runs[0].font.size = Pt(14)
        cell.paragraphs[0].runs[0].bold = True
        cell.paragraphs[0].runs[0].font.name = '黑体'
        cell.paragraphs[0].runs[0]._element.rPr.rFonts.set(qn('w:eastAsia'), '黑体')#中文要这样设置
        Set_Background_Color(cell,RGBColor(220,230,241))
        for j in range(1, data.shape[0] + 1):
            cell = table.cell(j, i)
            cell.text = str(data.iloc[j - 1, i])

            # 设置仿宋_GB2312小四字体
            cell.paragraphs[0].runs[0].bold = False
            cell.paragraphs[0].runs[0].font.name = '仿宋_GB2312'
            cell.paragraphs[0].runs[0]._element.rPr.rFonts.set(qn('w:eastAsia'), '仿宋_GB2312')#中文要这样设置
            cell.paragraphs[0].runs[0].font.size = Pt(12)
            if i!=0:
                cell.paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
                cell.paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    # 处理最后一行
    if tail:
        last_row = table.rows[-1]
        # 遍历最后一行的单元格
        for cell in last_row.cells:
            if cell.text=='总计':
            # 获取单元格的段落
                cell.paragraphs[0].paragraph_format.alignment=WD_ALIGN_PARAGRAPH.CENTER
                cell.paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                paragraph = cell.paragraphs[0]
                cell.paragraphs[0].runs[0]._element.rPr.rFonts.set(qn('w:eastAsia'), '黑体')#中文要这样设置
                cell.paragraphs[0].runs[0].font.size = Pt(14)
                cell.paragraphs[0].runs[0].bold = True
            Set_Background_Color(cell,RGBColor(220,230,241))
    document.add_paragraph().add_run('')#增加空行

# 数据处理

In [8]:
try:
    current_time = datetime.now()

    year = current_time.year
    month = current_time.month
    day = current_time.day
    graduate_year = month >8 and year+1 or year

    title='各性别就业率'
    data=data_group[key_group[title]]

    bysrs=int(xy(data,title,'合计','毕业人数小计'))
    qxlsl=change_to_decimal(xy(data,title,'合计','就业率小计'))
    boy_qxlsl=change_to_decimal(xy(data,title,'1男','就业率小计'))
    girl_qxlsl=change_to_decimal(xy(data,title,'2女','就业率小计'))

    zk_bysrs=int(xy(data,title,'合计','专科毕业人数'))
    zk_jyrs=int(xy(data,title,'合计','专科就业人数'))
    zk_qxlsl=change_to_decimal(xy(data,title,'合计','专科就业率'))
    zk_boy_qxlsl=change_to_decimal(xy(data,title,'1男','专科就业率'))
    zk_girl_qxlsl=change_to_decimal(xy(data,title,'2女','专科就业率'))
    bk_bysrs=int(xy(data,title,'合计','本科毕业人数'))
    bk_jyrs=int(xy(data,title,'合计','本科就业人数'))
    bk_qxlsl=change_to_decimal(xy(data,title,'合计','本科就业率'))
    bk_boy_qxlsl=change_to_decimal(xy(data,title,'1男','本科就业率'))
    bk_girl_qxlsl=change_to_decimal(xy(data,title,'2女','本科就业率'))
    ss_bysrs=int(xy(data,title,'合计','硕士生毕业人数'))
    ss_jyrs=int(xy(data,title,'合计','硕士生就业人数'))
    ss_qxlsl=change_to_decimal(xy(data,title,'合计','硕士生就业率'))
    ss_boy_qxlsl=change_to_decimal(xy(data,title,'1男','硕士生就业率'))
    ss_girl_qxlsl=change_to_decimal(xy(data,title,'2女','硕士生就业率'))
    bs_bysrs=int(xy(data,title,'合计','博士生毕业人数'))
    bs_jyrs=int(xy(data,title,'合计','博士生就业人数'))
    bs_qxlsl=change_to_decimal(xy(data,title,'合计','博士生就业率'))
    bs_boy_qxlsl=change_to_decimal(xy(data,title,'1男','博士生就业率'))
    bs_girl_qxlsl=change_to_decimal(xy(data,title,'2女','博士生就业率'))

    title='各生源地（省）就业率'
    data=data_group[key_group[title]]
    data.head()

    zk_hn_jybl=change_to_decimal(xy(data,title,'43湖南省','专科就业率'))
    bk_hn_jybl=change_to_decimal(xy(data,title,'43湖南省','本科就业率'))
    ss_hn_jybl=change_to_decimal(xy(data,title,'43湖南省','硕士生就业率'))
    bs_hn_jybl=change_to_decimal(xy(data,title,'43湖南省','博士生就业率'))
    hn_jybl=change_to_decimal(xy(data,title,'43湖南省','就业率小计'))

    title='就业地区东中西分布'
    bl_name='各分部占比.4'
    data=data_group[key_group[title]][[title,bl_name]].sort_values(by=bl_name,ascending=False).reset_index(drop=True)
    jydq1_name=data.loc[1][title]
    jydq2_name=data.loc[2][title]
    jydq3_name=data.loc[3][title]
    jydq_bl1=change_to_decimal(data.loc[1][bl_name])
    jydq_bl2=change_to_decimal(data.loc[2][bl_name])

    title='就业地区省份分布'
    bl_name='各分部占比.4'
    data=data_group[key_group[title]][[title,bl_name]].sort_values(by=bl_name,ascending=False).reset_index(drop=True)
    jydqsf1_name=re.sub('\d','',data.loc[1][title])
    jydqsf2_name=re.sub('\d','',data.loc[2][title])
    jydqsf_bl1=change_to_decimal(data.loc[1][bl_name])
    jydqsf_bl2=change_to_decimal(data.loc[2][bl_name])

    title='就业单位性质分布'
    bl_name='各分部占比.4'
    data=data_group[key_group[title]][[title,bl_name]].sort_values(by=bl_name,ascending=False).reset_index(drop=True)

    jydwxz1_name=re.sub('\d','',data.loc[1][title])
    jydwxz2_name=re.sub('\d','',data.loc[2][title])
    jydwxz3_name=re.sub('\d','',data.loc[3][title])
    jydwxz4_name=re.sub('\d','',data.loc[4][title])
    jydwxz_bl1=change_to_decimal(data.loc[1][bl_name])
    jydwxz_bl2=change_to_decimal(data.loc[2][bl_name])

    title='职位类型分布'
    bl_name='各分部占比.4'
    data=data_group[key_group[title]][[title,bl_name]].sort_values(by=bl_name,ascending=False).reset_index(drop=True)
    # display(data)
    jyzwlx1_name=re.sub('\d','',data.loc[1][title])
    jyzwlx2_name=re.sub('\d','',data.loc[2][title])
    jyzwlx_bl1=change_to_decimal(data.loc[1][bl_name])
    jyzwlx_bl2=change_to_decimal(data.loc[2][bl_name])

    title='就业单位行业分布'
    bl_name='各分部占比.4'
    data=data_group[key_group[title]][[title,bl_name]].sort_values(by=bl_name,ascending=False).reset_index(drop=True)
    # display(data)
    jydwhy1_name=re.sub('\d','',data.loc[1][title])
    jydwhy2_name=re.sub('\d','',data.loc[2][title])
    jydwhy_bl1=change_to_decimal(data.loc[1][bl_name])
    jydwhy_bl2=change_to_decimal(data.loc[2][bl_name])
    print('程序数据处理成功')
except:
    print('程序数据处理报错，错误信息如下：---------------------------')
    traceback.print_exc()
    time.sleep(5)
    raise
    

程序数据处理成功


# 替换word模板

In [9]:
try:
    document=Document(model_path+'/模板.docx')
    replace_name=['year','month','day',
    'bysrs',
     'qxlsl',
     'boy_qxlsl',
     'girl_qxlsl',
     'hn_jybl',
     'zk_bysrs',
     'zk_jyrs',
     'zk_qxlsl',
     'zk_boy_qxlsl',
     'zk_boy_qxlsl',
     'zk_hn_jybl',
     'bk_bysrs',
     'bk_jyrs',
     'bk_qxlsl',
     'bk_boy_qxlsl',
     'bk_boy_qxlsl',
     'bk_hn_jybl',
     'ss_bysrs',
     'ss_jyrs',
     'ss_qxlsl',
     'ss_boy_qxlsl',
     'ss_boy_qxlsl',
     'ss_hn_jybl',
     'bs_bysrs',
     'bs_jyrs',
     'bs_qxlsl',
     'bs_boy_qxlsl',
     'bs_boy_qxlsl',
     'bs_hn_jybl',
     'jydq1_name',
     'jydq_bl1',
     'jydq2_name',
     'jydq_bl2',
     'jydq3_name',
     'jydqsf1_name',
     'jydqsf_bl1',
     'jydqsf2_name',
     'jydqsf_bl2',
     'jydwxz1_name',
     'jydwxz_bl1',
     'jydwxz2_name',
     'jydwxz_bl2',
     'jydwxz3_name',
     'jydwxz4_name',
     'jyzwlx1_name',
     'jyzwlx_bl1',
     'jyzwlx2_name',
     'jyzwlx_bl2',
     'jydwhy1_name',
     'jydwhy_bl1',
     'jydwhy2_name',
     'jydwhy_bl2']

    for j in range(len(document.paragraphs)):
        flag=False
        if len(document.paragraphs[j].runs)>0:#先获取run的格式，不然后面replace似乎就没有格式了
            fname=document.paragraphs[j].runs[0].font.name
            fsize=document.paragraphs[j].runs[0].font.size
        for i in replace_name:
            if '【'+i+'】' in document.paragraphs[j].text:
                flag=True
                document.paragraphs[j].text = document.paragraphs[j].text.replace('【'+i+'】',str(eval(i)))
        if flag:
            for run in document.paragraphs[j].runs:
                run.font.name=fname
                run._element.rPr.rFonts.set(qn('w:eastAsia'), fname)
                run.font.size=fsize
    print('程序替换word模板成功！')
except:
    print('程序替换word模板报错，错误信息如下：---------------------------')
    traceback.print_exc()
    time.sleep(5)
    raise
    

程序替换word模板成功！


# 八、 访企拓岗总体情况

In [12]:
try:

#     cookies = {
#         'JSESSIONID': '6B86C083DCCB90C39B47E825FE1360BE',
#         'CHSICC_CLIENTFLAGJYWEB': '917451e1e28a97544d747c79c404336d',
#         'CHSICC01': '!6ecBcjTvitfqdTXzYxYLahOzddj6Y9N0NWOVI9Io3STyY4/aBFrriqH5qdCycGjlabioM2vAvYqC',
#         'Hm_lvt_fc3f3af5c2ccc8d560855e956adb15ca': '1689735070',
#         'Hm_lpvt_fc3f3af5c2ccc8d560855e956adb15ca': '1689735423',
#     }

#     headers = {
#         'Accept': '*/*',
#         'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
#         'Connection': 'keep-alive',
#         # 'Cookie': 'JSESSIONID=6B86C083DCCB90C39B47E825FE1360BE; CHSICC_CLIENTFLAGJYWEB=917451e1e28a97544d747c79c404336d; CHSICC01=!6ecBcjTvitfqdTXzYxYLahOzddj6Y9N0NWOVI9Io3STyY4/aBFrriqH5qdCycGjlabioM2vAvYqC; Hm_lvt_fc3f3af5c2ccc8d560855e956adb15ca=1689735070; Hm_lpvt_fc3f3af5c2ccc8d560855e956adb15ca=1689735423',
#         'Referer': f'https://jy.ncss.cn/fqtgxd/{year}/index.html',
#         'Sec-Fetch-Dest': 'empty',
#         'Sec-Fetch-Mode': 'cors',
#         'Sec-Fetch-Site': 'same-origin',
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.82',
#         'X-Requested-With': 'XMLHttpRequest',
#         'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Microsoft Edge";v="114"',
#         'sec-ch-ua-mobile': '?0',
#         'sec-ch-ua-platform': '"Windows"',
#     }

#     params = {
#         'year': str(year),
#         'yxdm': '',
#         'fxmc': '',
#         '_': '1689735619762',
#     }

#     response = requests.get('https://jy.ncss.cn/fqtgxd/statistics', params=params, cookies=cookies, headers=headers)

#     school_list=response.json()['provinceCountVOList']

#     school_num=0
#     company_num=0
#     for i in school_list:
#         if i['count']!=0:
#             school_num+=1
#             company_num+=i['count']
    
    title='username'
    data=data_group[key_group[title]][['qa3_1','qa3_2','qa5_1']]
    data['qa3_1']=data['qa3_1'].apply(lambda x:x if x>0 else 0)
    data['qa3_2']=data['qa3_2'].apply(lambda x:x if x>0 else 0)
    data['qa5_1']=data['qa5_1'].apply(lambda x:x if x>0 else 0)
    data['temp']=data['qa5_1'].apply(lambda x:1 if x>0 else 0)
    xzjygw=int(sum(data['qa3_1'])+sum(data['qa3_2']))
    sxsjjd=int(sum(data['temp']))
    xjnsxs=int(sum(data['qa5_1']))

    content1=f"\t 八、 访企拓岗总体情况"
    content2=f"截至{year}年{month}月{day}日9点整，为促进{month>8 and year+1 or year}届毕业生就业，全省共有{school_num}所高校开展访企拓岗活动，已走访{company_num}家单位，促成企业新增就业岗位{xzjygw}个，新签订{sxsjjd}家实习实践基地、拟新接纳实习生{xjnsxs}人。"
    add_p(content1,content2)
    
    print('访企拓岗导入成功！')
except:
    print('访企拓岗报错，错误信息如下：---------------------------')
    traceback.print_exc()
    document.save(f'高校毕业生就业数据分析简报.docx')
    time.sleep(5)
    raise
    

访企拓岗报错，错误信息如下：---------------------------


Traceback (most recent call last):
  File "d:\python3.8\lib\site-packages\requests\models.py", line 971, in json
    return complexjson.loads(self.text, **kwargs)
  File "d:\python3.8\lib\json\__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "d:\python3.8\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "d:\python3.8\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 22 column 1 (char 42)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\龙辛柯\AppData\Local\Temp\ipykernel_2412\215296126.py", line 36, in <module>
    school_list=response.json()['provinceCountVOList']
  File "d:\python3.8\lib\site-packages\requests\models.py", line 975, in json
    raise RequestsJSONDecodeError(e.msg, e.doc, e.pos)
requests.exceptions.JSOND

JSONDecodeError: Expecting value: line 22 column 1 (char 42)

# 导出

In [42]:
try:
    document.save(f'高校毕业生就业数据分析简报.docx')
except:
    print('导出报错，错误信息如下：---------------------------')
    traceback.print_exc()
    time.sleep(5)
    raise

In [ ]:
print('程序结束')
time.sleep(5)